In [0]:
from __future__ import unicode_literals
from sklearn.pipeline import Pipeline
from sklearn.decomposition import SparsePCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import numpy as np
import unicodedata
import nltk
import scipy
import urllib
%matplotlib inline

In [2]:
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /content/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /content/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /content/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /content/nltk_data...
[nltk_data]    |   Unzipping corpora/na

True

## Exploratory data analysis

###Loading the data

In [0]:
#url = "https://raw.githubusercontent.com/mkobbi/subvention-status-datacamp/master/data/subventions-accordees-et-refusees.csv"
#url_departements = "http://sql.sh/ressources/sql-departement-france/departement.csv"
#data = pd.read_csv(url, sep=";")
train_filename = 'data/train.csv'
train = pd.read_csv(train_filename, sep=';')
test = pd.read_csv('data/test.csv', sep=';')
frames = [train, test]
data = pd.concat(frames, axis=0)
fr_stopwords_url = "https://raw.githubusercontent.com/mkobbi/subvention-status-datacamp/master/data/stopwords-filter-fr.txt"
#departements = pd.read_csv(url_departements, sep=",", names=["code","full_name"], usecols=[0,3])#[["full_name","code"]]
#departements.set_index("full_name").to_dict()["code"]

In [4]:
string_columns = ["Nom du partenaire", "Intitulé de la demande"]
to_drop_columns = ["Année", "Siret", "N° SIMPA","CP-Adresse-Libellé voie", "CP-Adresse-Ville"] #CP-Adresse-Ville se vota porque da la misma informacion que la columna CP-Adresse-Code postal
str_categorical_columns = ["Nom du partenaire", "Appel à projets","Appel à projets PolVille"]
#num_categorical_columns = ["Année", "CP-Adresse-Code postal"]
num_categorical_columns = ["CP-Adresse-Code postal"]
#data['Année'] = pd.to_datetime(data['Année'])
data = data.fillna(value=0,axis='columns')
data[string_columns] = data[string_columns].apply(lambda x: x.str.upper().str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8'))
data[str_categorical_columns] = data[str_categorical_columns].apply(lambda x: x.astype('category').cat.codes)
data[num_categorical_columns] = data[num_categorical_columns].apply(lambda x: x.astype('int'))#.astype('category'))
fr_stopwords = urllib.request.urlopen(fr_stopwords_url).read().decode("utf-8").upper()
fr_stopwords = fr_stopwords.split("\n")
#print(fr_stopwords)
data = data.drop(to_drop_columns, axis='columns')
data.head()


,Nom du partenaire,Intitulé de la demande,Appel à projets,Montant voté par demande,Fonctionnement,Subventions sur projet,Subventions d'équipement,Subventions sans nature spécifiée,CP-Adresse-Code postal,Appel à projets PolVille,S-DE-Montant demandé,Total voté
0,3791,FONCTIONNEMENT 2013,0,183622.0,183622.0,0.0,0.0,0.0,75014,0,184782.0,183622.0
1,3792,SUBVENTION DE FONCTIONNEMENT MAISON DES CONTES...,0,1500.0,1500.0,0.0,0.0,0.0,75004,0,10000.0,1500.0
2,3796,ACCOMPAGNEMENT AU PASSAGE DU CODE DU PERMIS B ...,0,11000.0,0.0,11000.0,0.0,0.0,75010,1,13500.0,11000.0
3,3796,AIDE DEGRESSIVE AU DEMARRAGE DE LA REGIE DE QU...,0,20000.0,20000.0,0.0,0.0,0.0,75010,0,30000.0,20000.0
4,3902,DEMANDE DE SUBVENTION D AIDE AU FONCTIONNEMENT...,0,2000.0,2000.0,0.0,0.0,0.0,75004,0,4750.0,2000.0


In [0]:
print(np.unique(data['Nom du partenaire']))
print(len(np.unique(data['state'])))
data.groupby('state').count()[['job']].sort_values(
    'job', ascending=False).reset_index().rename(
    columns={'job': 'count'}).plot.bar(
    x='state', y='count', figsize=(16, 10), fontsize=18);

In [34]:
train_filename = 'data/train.csv'
train = pd.read_csv(train_filename, sep=';')
test = pd.read_csv('data/test.csv', sep=';')
frames = [train, test]
data = pd.concat(frames, axis=0)

FileNotFoundError: ignored

In [5]:
data.dtypes

Nom du partenaire                      int16
Intitulé de la demande                object
Appel à projets                         int8
Montant voté par demande             float64
Fonctionnement                       float64
Subventions sur projet               float64
Subventions d'équipement             float64
Subventions sans nature spécifiée    float64
CP-Adresse-Code postal                 int64
Appel à projets PolVille                int8
S-DE-Montant demandé                 float64
Total voté                           float64
dtype: object

In [0]:
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
data['stemmed'] = data['Intitulé de la demande'].map(lambda x: ' '.join([stemmer.stem(y) for y in x.split(' ')]))
#data.stemmed.head()

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

#cvec = CountVectorizer(stop_words='french', min_df=1, max_df=.5, ngram_range=(1,2))
#cvec
vect = TfidfVectorizer(norm='l2',min_df=0, max_df=1, use_idf=True, smooth_idf=True, 
                             sublinear_tf=True, stop_words=fr_stopwords, analyzer='word')
words = vect.fit_transform(data.stemmed)
words = pd.DataFrame(words.todense(),columns=vect.get_feature_names())
data = pd.concat([data, words], axis='columns')
data = data.drop(["Intitulé de la demande", "stemmed"],axis='columns')


In [0]:
#X = scipy.sparse.csr_matrix(data.values)
X = normalize(scipy.sparse.csr_matrix(data.values))

In [0]:
y = np.ravel(pd.DataFrame([data["Total voté"] > 0.0]).astype(int))

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

## Classifying

In [14]:
clf = SVC(kernel = 'linear', class_weight = 'balanced', random_state = 42)
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
y_pred

array([1, 0, 0, ..., 1, 0, 1])

In [15]:
accuracy_score(y_test,y_pred)

0.952054794520548

In [18]:
data.describe()

,Nom du partenaire,Appel à projets,Montant voté par demande,Fonctionnement,Subventions sur projet,Subventions d'équipement,Subventions sans nature spécifiée,Appel à projets PolVille,S-DE-Montant demandé,Total voté
count,26273.000000,26273.000000,2.627300e+04,2.627300e+04,2.627300e+04,2.627300e+04,26273.000000,26273.000000,2.627300e+04,2.627300e+04
mean,2927.560499,0.390287,4.283378e+04,3.532798e+04,2.966839e+03,3.334278e+03,556.839303,0.177445,5.668498e+04,2.917313e+04
std,1745.030637,0.487824,4.137719e+05,3.929649e+05,4.238138e+04,8.094521e+04,13788.911899,0.382052,4.958573e+05,3.479400e+05
min,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,1434.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,4.000000e+03,0.000000e+00
50%,2849.000000,0.000000,2.500000e+03,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,8.000000e+03,0.000000e+00
75%,4439.000000,1.000000,8.740000e+03,4.000000e+03,0.000000e+00,0.000000e+00,0.000000,0.000000,2.000000e+04,5.000000e+03
max,6068.000000,1.000000,1.744600e+07,1.744600e+07,5.947000e+06,7.599654e+06,1000000.000000,1.000000,3.915823e+07,1.744600e+07


In [9]:
data.count()

Année                                26273
Nom du partenaire                    26273
Intitulé de la demande               26273
Appel à projets                      26273
Montant voté par demande             26273
Fonctionnement                       26273
Subventions sur projet               26273
Subventions d'équipement             26273
Subventions sans nature spécifiée    26273
CP-Adresse-Code postal               26273
Appel à projets PolVille             26273
S-DE-Montant demandé                 26273
Total voté                           26273
dtype: int64

The dataset contains 26000+ instances mostly numericals and one categorical feature.